# Distribución Metros Cuadrados Construidos


In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "tulancingo" in nombre    
    and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_q_jul_2022_tulancingo',
 'df_alfa_q_jul_2023_tulancingo',
 'df_alfa_q_jun_2023_tulancingo',
 'df_alfa_q_mar_2023_tulancingo',
 'df_alfa_q_may_2022_tulancingo',
 'df_alfa_q_may_2023_tulancingo',
 'df_alfa_q_oct_2022_tulancingo',
 'df_alfa_q_oct_2023_tulancingo',
 'df_alfa_q_sep_2023_tulancingo']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
dataframes_name = []

for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'categoria','m2_construido']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
     # Componer el nombre del DataFrame en la lista dataframes_name
    dataframes_name.append(nombre_df)


# Cantidad de registros por metros cuadrados construidos, por segmento

In [4]:
def procesar_dataframes(dataframes_list, dataframes_name):
    for i, df in enumerate(dataframes_list):
        # Crear una nueva columna con la primera letra de la categoría
        df['segmento'] = df['categoria'].str[0]
        df.drop(columns=['categoria'], inplace=True)

        # Clasificar 'baño_total' en rangos específicos
        bins = [0, 100, 150, 200, 250, 300, float('inf')]
        labels = ['0-100', '100-150', '150-200', '200-250', '250-300', '300+']
        df['m2_construido'] = pd.cut(df['m2_construido'], bins=bins, labels=labels, right=False)

        # Agrupar y contar los datos
        tabla = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)

        # Imprimir el nombre del DataFrame y la tabla resultante
        print(f"Nombre del DataFrame: {dataframes_name[i]}")
        print(tabla)

# Procesar los DataFrames de la lista
procesar_dataframes(dataframes_list, dataframes_name)

Nombre del DataFrame: df_alfa_q_jul_2022_tulancingo
m2_construido  0-100  100-150  150-200  200-250  250-300  300+
segmento                                                      
A                  0        0        0        0        1     1
B                  0        2        2        3        0     0
C                  1        5       16        0        2     0
D                 12       16        3        1        1     1
E                  9        1        0        0        0     0
S                  0        0        0        0        0     1
Nombre del DataFrame: df_alfa_q_jul_2023_tulancingo
m2_construido  0-100  100-150  150-200  200-250  250-300  300+
segmento                                                      
A                  0        1        0        2        0     1
B                  0        2        2        3        0     0
C                  3       11        2        1        0     2
D                 13       18        2        3        1     2
E             

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\275059014.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\275059014.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['categoria'], inplace=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\275059014.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

## Tabla general de frecuencia
 Recuento total de filas para cada combinación de valores únicos de 'segmento' y 'm2_contruido' con todos los DataFrame

In [5]:
# Crear una nueva función para sumar las tablas de todos los DataFrames
def sumar_tablas(dataframes_list):
    # Inicializar una tabla vacía para almacenar la suma
    tabla_suma = None
    
    # Iterar sobre cada DataFrame en la lista
    for df in dataframes_list:
        # Agrupar y contar los datos en el DataFrame actual
        tabla_actual = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)
        
        # Sumar la tabla actual al acumulador (tabla_suma)
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    # Imprimir la tabla resultante de la suma
    print("Tabla general de frecuencia:")
    print(tabla_suma)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
sumar_tablas(dataframes_list)

Tabla general de frecuencia:
m2_construido  0-100  100-150  150-200  200-250  250-300  300+
segmento                                                      
A                  0        3       12       22        6    15
B                  1       19       19       29        4     3
C                 14       62       62       20        6    14
D                160      162       30       16        3    10
E                112       21       10        6       10     3
S                  0        0       13       25        5    54


C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\470711682.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\470711682.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\470711682.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain curren

In [6]:

def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_bar_dist_m2_construidos', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):  
    segmentos = tabla.index
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, segmento in enumerate(segmentos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=banos,
            y=tabla.loc[segmento],
            name=segmento,
            marker_color=color
        ))
    
    fig.update_layout(
        xaxis_title="Segmento",        
        yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda',
            gridwidth=1,
        ),
        legend_title="Rango de M2",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
    )    
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    ruta_archivo = os.path.join(carpeta, f'{nombre_archivo}.html')
    pio.write_html(fig, ruta_archivo)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)


C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\403224666.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\403224666.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\403224666.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain curren

Tabla general de frecuencia:
segmento        A   B   C    D    E   S
m2_construido                          
0-100           0   1  14  160  112   0
100-150         3  19  62  162   21   0
150-200        12  19  62   30   10  13
200-250        22  29  20   16    6  25
250-300         6   4   6    3   10   5
300+           15   3  14   10    3  54


C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\403224666.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\403224666.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\403224666.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain curren

In [7]:
import os
import plotly.io as pio

def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_pie_dist_m2_construidos', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):
    # Sumar los valores por 'm2_contruido' para obtener el total por rango de M2
    suma_por_m2 = tabla.sum(axis=1)
    
    fig = go.Figure(data=[go.Pie(
        labels=suma_por_m2.index.astype(str),
        values=suma_por_m2.values,
        marker=dict(colors=colores),
        textinfo='label+percent',
        insidetextorientation='radial'
    )])
    
    fig.update_layout(
        legend_title="Rango de M2",
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02, 
            xanchor='right',
            x=0.72
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)
    )    
    
    fig.show()
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    ruta_archivo = os.path.join(carpeta, f'{nombre_archivo}.html')
    pio.write_html(fig, ruta_archivo)

fig1 = sumar_tablas(dataframes_list)


Tabla general de frecuencia:
segmento        A   B   C    D    E   S
m2_construido                          
0-100           0   1  14  160  112   0
100-150         3  19  62  162   21   0
150-200        12  19  62   30   10  13
200-250        22  29  20   16    6  25
250-300         6   4   6    3   10   5
300+           15   3  14   10    3  54


C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\4102199724.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\4102199724.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\4102199724.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16216\4102199724.py:8: FutureWarning:

Th

import shutil

archivo_origen = 'graficas/g_pie_dist_m2_construidos.html'
nuevo_directorio = 'media/graficas'

if not os.path.exists(nuevo_directorio):
    os.makedirs(nuevo_directorio)

nuevo_archivo = os.path.join(nuevo_directorio, 'g_pie_dist_m2_construidos.html')
shutil.move(archivo_origen, nuevo_archivo)
print(f"Archivo copiado en: {nuevo_archivo}")
import shutil

archivo_origen = 'graficas/g_bar_dist_m2_construidos.html'
nuevo_directorio = 'media/graficas'

if not os.path.exists(nuevo_directorio):
    os.makedirs(nuevo_directorio)

nuevo_archivo = os.path.join(nuevo_directorio, 'g_bar_dist_m2_construidos.html')
shutil.move(archivo_origen, nuevo_archivo)
print(f"Archivo copiado en: {nuevo_archivo}")